### import data in floor

In [2]:
from pandas import DataFrame, Series
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [1]:
bld1_02 = pd.read_csv('/Users/AAA218/Desktop/ami_sg/SDE.csv/SDE1_02.csv',header = 0, sep=",",low_memory=False)
bld1_03 = pd.read_csv('/Users/AAA218/Desktop/ami_sg/SDE.csv/SDE1_03.csv',header = 0, sep=",",low_memory=False)
bld1_04 = pd.read_csv('/Users/AAA218/Desktop/ami_sg/SDE.csv/SDE1_04.csv',header = 0, sep=",",low_memory=False)
bld1_05 = pd.read_csv('/Users/AAA218/Desktop/ami_sg/SDE.csv/SDE1_05.csv',header = 0, sep=",",low_memory=False)
bld2_01 = pd.read_csv('/Users/AAA218/Desktop/ami_sg/SDE.csv/SDE2_01.csv',header = 0, sep=",",low_memory=False)
bld2_02 = pd.read_csv('/Users/AAA218/Desktop/ami_sg/SDE.csv/SDE2_02.csv',header = 0, sep=",",low_memory=False)
bld2_03 = pd.read_csv('/Users/AAA218/Desktop/ami_sg/SDE.csv/SDE2_03.csv',header = 0, sep=",",low_memory=False)
bld3_01 = pd.read_csv('/Users/AAA218/Desktop/ami_sg/SDE.csv/SDE3_01.csv',header = 0, sep=",",low_memory=False)
bld3_02 = pd.read_csv('/Users/AAA218/Desktop/ami_sg/SDE.csv/SDE3_02.csv',header = 0, sep=",",low_memory=False)
bld3_03 = pd.read_csv('/Users/AAA218/Desktop/ami_sg/SDE.csv/SDE3_03.csv',header = 0, sep=",",low_memory=False)
bld3_04 = pd.read_csv('/Users/AAA218/Desktop/ami_sg/SDE.csv/SDE3_04.csv',header = 0, sep=",",low_memory=False)

In [8]:
def diff(col):
    t_fun = lambda x : datetime.strptime(x[0:19],'%Y-%m-%dT%H:%M:%S')
    secs = []
    fir_t = []
    las_t = []
    for i, row in enumerate(col.values):
        fir = t_fun(col.firstlocatedtime[i])
        las = t_fun(col.lastlocatedtime[i])
        dif = timedelta.total_seconds(las - fir)
        print dif
        secs.append(dif)
        fir_t.append(fir)
        las_t.append(las)
    print "interval len: %s" %len(secs)
    col['firstlocatedtime'] = fir_t
    col['lastlocatedtime'] = las_t
    col['interval'] = secs
    return col

In [61]:
import glob, os
ans = raw_input('Please enter the full directory for files:')
try:
    dirname = str(ans)
    os.chdir(dirname)
except:
    dirname = "/Users/AAA218/Desktop/ami_sg/SDE.csv/"
    print'not valid, using defaults: %s' % dirname
    os.chdir(dirname)
l = list()
for file in glob.glob("*.csv"):
    if file.startswith('SDE'):
        l.append(file)

Please enter the full directory for files:7
not valid, using defaults: /Users/AAA218/Desktop/ami_sg/SDE.csv/


In [62]:
dim = {"SDE1-02":[1227.0858,1730.8369],
"SDE1-03":[275.0,394.0],
"SDE1-04":[1065.6272,1503.0952],
"SDE1-05":[1227.0858,1730.8369],
"SDE2-01":[671.634,948.0622],
"SDE2-02":[736.25146,1038.5022],
"SDE2-03":[673.7935,951.3015],
"SDE3-01" :[1214.772,1734.6945],
"SDE3-02": [2018.2332,2882.037],
"SDE3-03" :[1345.4888,1921.358],
"SDE3-04":[683.8,474.7]} #[length,width]=> [ycor,xcor]

### writedb.py testing
* read csv repeatively and write to db with preprocessing

In [63]:
def out_dim(s):
    bld = str(s["bldid"])
    if(0< s["xcor"] <= dim[bld][1] and 0< s["ycor"] <= dim[bld][0]):
            return pd.Series(dict(out=0))
    else: 
            return pd.Series(dict(out=1))

In [78]:
names = ["sgtime", "mac","guestid","status", "bldid", "xcor", "ycor","currenttime","firstlocatedtime","lastlocatedtime"]
# function to create interval
def diff(col):
    t_fun = lambda x : datetime.strptime(x[0:19],'%Y-%m-%dT%H:%M:%S')
    fir = col['firstlocatedtime'].map(t_fun)
    las = col['lastlocatedtime'].map(t_fun)
    dif = las - fir
    dif_sec = dif.map(timedelta.total_seconds)
    col['firstlocatedtime'] = fir
    col['lastlocatedtime'] = las
    col['interval'] = dif_sec  
    return col
import sqlite3
conn = sqlite3.connect('new.sqlite')
for f in l:
    fname = dirname + f
    #index_start = 1
    for df in pd.read_csv(fname, header = 0,sep=',',low_memory= False,iterator=True):
        #print df.columns
        df.columns = names
        #print "new_____", df.sgtime[0:10]
        try: 
            df['sgtime'] = pd.to_datetime(df["sgtime"])
            print "parse the date and make the interval"
            diff(df)
        except:
            print "skip file: %s" % fname
            continue
        print "reformat the building id...."
        bld = df["bldid"].apply(lambda x: x[-7:]).copy()
        df['bldid'] = bld
        print "create Out dim mark"
        st1 = datetime.now()
        df['out'] = df.apply(out_dim , axis = 1)
        st2 = datetime.now()
        print "used %s sec.."%timedelta.total_seconds(st2 - st1)
        #df.status.replace({'PROBING','ASSOCIATED','UNKNOWN'},value= {0,1,-1},inplace=True)
        df.drop(['guestid','currenttime'], axis=1, inplace=True)
        print 'write to db %s' %f
        df.to_sql(name='wifi',index=False,con=conn,if_exists='append',dtype={"sgtime": "DATETIME", "status":"INTEGER", "mac": "CHAR(20)","bldid": "CHAR(20)"," xcor ":"NUMERIC","ycor":" NUMERIC","firstlocatedtime":" DATETIME","lastlocatedtime":"DATETIME","interval": "NUMERIC", "hrfir":"INTEGER","hrlas":"INTEGER", "out": "INTEGER"})

Index([u'sgTime', u'macAddress', u'isGuestUser', u'dot11Status',
       u'mapHierarchyString', u'MapCoordinate_X', u'MapCoordinate_Y',
       u'currentServerTime', u'firstLocatedTime', u'lastLocatedTime'],
      dtype='object')
new_____ 0    2016-04-16 00:00:01
1    2016-04-16 02:40:01
2    2016-04-16 02:45:01
3    2016-04-16 07:50:01
4    2016-04-16 07:55:01
5    2016-04-16 08:25:01
6    2016-04-16 09:35:02
7    2016-04-16 09:40:01
8    2016-04-16 09:55:01
9    2016-04-16 10:00:02
Name: sgtime, dtype: object
parse the date and make the interval
reformat the building id....
create Out dim mark
write to db SDE1_02.csv
Index([u'sgTime', u'macAddress', u'isGuestUser', u'dot11Status',
       u'mapHierarchyString', u'MapCoordinate_X', u'MapCoordinate_Y',
       u'currentServerTime', u'firstLocatedTime', u'lastLocatedTime'],
      dtype='object')
new_____ 0    2016-04-16 00:00:01
1    2016-04-16 00:00:01
2    2016-04-16 00:00:01
3    2016-04-16 00:00:01
4    2016-04-16 00:00:01
5    2016-04-

In [2]:
names = ["sgtime", "mac","guestid","status_info", "bldid", "xcor", "ycor","currentservertime","firstlocatedtime","lastlocatedtime"]
bld3_04.columns =bld3_03.columns =bld3_02.columns =bld3_01.columns =bld2_03.columns =bld2_02.columns =bld2_01.columns =bld1_05.columns =bld1_04.columns =bld1_03.columns =bld1_02.columns = names

In [3]:
print "1_03----", bld1_03[["xcor","ycor"]].describe()
print "1_04----", bld1_04[["xcor","ycor"]].describe()

1_03----                 xcor           ycor
count  358556.000000  358556.000000
mean      198.271213      79.756742
std        87.952440      36.294464
min        -4.000000      -3.990000
25%       130.160000      54.430000
50%       242.760000      88.410000
75%       264.570000     107.590000
max       394.000000     275.000000
1_04----                 xcor           ycor
count  260937.000000  260937.000000
mean      831.545927     531.740730
std       150.535272     167.033891
min       227.380000     265.910000
25%       774.040000     392.460000
50%       859.370000     457.800000
75%       947.340000     635.390000
max      1094.220000     886.160000


### Stack everybuilding together & reformat the building id

In [4]:
b1_23 = bld1_02.append([bld1_03,bld1_04,bld1_05],ignore_index=True).copy()
print b1_23.shape   #679922

(679922, 10)


In [5]:
b2_3 = b1_23.append([bld2_01,bld2_02,bld2_03,],ignore_index=True).copy()
print b2_3.shape    #1194183

(1194183, 10)


In [6]:
b3_4 = b2_3.append([bld3_01,bld3_02,bld3_03,bld3_04],ignore_index=True).copy()
print b3_4.shape    #2650553

(2650553, 10)


In [7]:
a = b3_4["bldid"].apply(lambda x: x[-7:]).copy()

In [8]:
set(a)

{'SDE1-02',
 'SDE1-03',
 'SDE1-04',
 'SDE1-05',
 'SDE2-01',
 'SDE2-02',
 'SDE2-03',
 'SDE3-01',
 'SDE3-02',
 'SDE3-03',
 'SDE3-04'}

In [9]:
b3_4["bldid"] = a
print set(b3_4["bldid"])

set(['SDE2-02', 'SDE2-03', 'SDE2-01', 'SDE3-04', 'SDE3-01', 'SDE3-03', 'SDE3-02', 'SDE1-05', 'SDE1-04', 'SDE1-03', 'SDE1-02'])


In [10]:
print set(b3_4["guestid"]),set(b3_4["status_info"]),len(set(b3_4["mac"]))
# 56918

set([False]) set(['PROBING', 'UNKNOWN', 'ASSOCIATED']) 56918


In [11]:
len(b3_4[b3_4["bldid"]== "SDE1-04"])
# 260937

260937

**********
#### Get the time intervals variable Diff

In [26]:
test_inx= b3_4.copy() # repeat this step if test column is dropped

In [16]:
fir = b3_4["firstlocatedtime"].copy()
las = b3_4["lastlocatedtime"].copy()
#fir1 = pd.to_datetime(fir, infer_datetime_format = True) #30s

In [27]:
test_inx.head()

,sgtime,mac,guestid,status_info,bldid,xcor,ycor,currentservertime,firstlocatedtime,lastlocatedtime
0,2016-04-16 00:00:01,e8:50:8b:94:67:3b,False,PROBING,SDE1-02,1588.27,382.58,2016-04-15T23:59:47.573+0800,2016-04-15T23:56:54.444+0800,2016-04-15T23:57:14.518+0800
1,2016-04-16 02:40:01,94:b1:0a:de:9b:ca,False,PROBING,SDE1-02,1588.27,382.58,2016-04-16T02:40:00.328+0800,2016-04-16T02:38:26.409+0800,2016-04-16T02:38:26.409+0800
2,2016-04-16 02:45:01,94:b1:0a:de:9b:ca,False,PROBING,SDE1-02,1588.27,382.58,2016-04-16T02:45:00.530+0800,2016-04-16T02:38:26.409+0800,2016-04-16T02:38:26.409+0800
3,2016-04-16 07:50:01,84:55:a5:70:d1:eb,False,PROBING,SDE1-02,1582.69,382.87,2016-04-16T07:49:57.232+0800,2016-04-16T07:48:14.620+0800,2016-04-16T07:48:14.620+0800
4,2016-04-16 07:55:01,84:55:a5:70:d1:eb,False,PROBING,SDE1-02,1582.69,382.87,2016-04-16T07:54:57.483+0800,2016-04-16T07:48:14.620+0800,2016-04-16T07:48:14.620+0800


In [19]:
datetime.strptime(test_inx.iloc[3][8][0:19],'%Y-%m-%dT%H:%M:%S')

datetime.datetime(2016, 4, 16, 7, 48, 14)

In [20]:
%%time
f = lambda x : datetime.strptime(x[0:19],'%Y-%m-%dT%H:%M:%S')
fir1 = fir.map(f)
las1 = las.map(f) #1min 21s

CPU times: user 1min 15s, sys: 398 ms, total: 1min 15s
Wall time: 1min 15s


In [21]:
print "last----",las1[1:10]
print "first----",fir1[1:10]

last---- 1   2016-04-16 02:38:26
2   2016-04-16 02:38:26
3   2016-04-16 07:48:14
4   2016-04-16 07:48:14
5   2016-04-16 08:24:32
6   2016-04-16 09:34:45
7   2016-04-16 09:34:45
8   2016-04-16 09:54:49
9   2016-04-16 09:54:49
Name: lastlocatedtime, dtype: datetime64[ns]
first---- 1   2016-04-16 02:38:26
2   2016-04-16 02:38:26
3   2016-04-16 07:48:14
4   2016-04-16 07:48:14
5   2016-04-16 08:24:32
6   2016-04-16 09:34:45
7   2016-04-16 09:34:45
8   2016-04-16 09:54:49
9   2016-04-16 09:54:49
Name: firstlocatedtime, dtype: datetime64[ns]


In [22]:
diff = las1-fir1
print diff.describe()

count                   2650553
mean     0 days 02:43:43.408063
std      0 days 16:19:50.322887
min             0 days 00:00:00
25%             0 days 00:00:20
50%             0 days 00:12:52
75%             0 days 01:15:59
max            11 days 07:35:28
dtype: object
2650548   00:12:14
2650549   00:00:00
2650550   01:26:16
2650551   00:00:08
2650552   00:00:00
dtype: timedelta64[ns]


In [23]:
dtest = diff.map(timedelta.total_seconds)
print dtest[1:5]
print dtest.describe()

1    0.0
2    0.0
3    0.0
4    0.0
dtype: float64


#### make a new column 'interval'

In [32]:
test_inx.loc[:,'interval']=dtest

In [33]:
test_inx.head()

,sgtime,mac,guestid,status_info,bldid,xcor,ycor,currentservertime,firstlocatedtime,lastlocatedtime,interval
0,2016-04-16 00:00:01,e8:50:8b:94:67:3b,False,PROBING,SDE1-02,1588.27,382.58,2016-04-15T23:59:47.573+0800,2016-04-15T23:56:54.444+0800,2016-04-15T23:57:14.518+0800,20.0
1,2016-04-16 02:40:01,94:b1:0a:de:9b:ca,False,PROBING,SDE1-02,1588.27,382.58,2016-04-16T02:40:00.328+0800,2016-04-16T02:38:26.409+0800,2016-04-16T02:38:26.409+0800,0.0
2,2016-04-16 02:45:01,94:b1:0a:de:9b:ca,False,PROBING,SDE1-02,1588.27,382.58,2016-04-16T02:45:00.530+0800,2016-04-16T02:38:26.409+0800,2016-04-16T02:38:26.409+0800,0.0
3,2016-04-16 07:50:01,84:55:a5:70:d1:eb,False,PROBING,SDE1-02,1582.69,382.87,2016-04-16T07:49:57.232+0800,2016-04-16T07:48:14.620+0800,2016-04-16T07:48:14.620+0800,0.0
4,2016-04-16 07:55:01,84:55:a5:70:d1:eb,False,PROBING,SDE1-02,1582.69,382.87,2016-04-16T07:54:57.483+0800,2016-04-16T07:48:14.620+0800,2016-04-16T07:48:14.620+0800,0.0


### Make timestamp | Add building id as index

In [34]:
t1 = b3_4["sgtime"].copy()
ts = pd.to_datetime(t1,infer_datetime_format=True)

In [ ]:
test_inx.drop(['sgtime','guestid','currentservertime'], axis=1, inplace=True)

In [ ]:
test_inx.index = ts

In [37]:
test_inx.head() #drop guestid, sgtime, current_time in column

,mac,status_info,bldid,xcor,ycor,firstlocatedtime,lastlocatedtime,interval
sgtime,,,,,,,,
2016-04-16 00:00:01,e8:50:8b:94:67:3b,PROBING,SDE1-02,1588.27,382.58,2016-04-15T23:56:54.444+0800,2016-04-15T23:57:14.518+0800,20.0
2016-04-16 02:40:01,94:b1:0a:de:9b:ca,PROBING,SDE1-02,1588.27,382.58,2016-04-16T02:38:26.409+0800,2016-04-16T02:38:26.409+0800,0.0
2016-04-16 02:45:01,94:b1:0a:de:9b:ca,PROBING,SDE1-02,1588.27,382.58,2016-04-16T02:38:26.409+0800,2016-04-16T02:38:26.409+0800,0.0
2016-04-16 07:50:01,84:55:a5:70:d1:eb,PROBING,SDE1-02,1582.69,382.87,2016-04-16T07:48:14.620+0800,2016-04-16T07:48:14.620+0800,0.0
2016-04-16 07:55:01,84:55:a5:70:d1:eb,PROBING,SDE1-02,1582.69,382.87,2016-04-16T07:48:14.620+0800,2016-04-16T07:48:14.620+0800,0.0


## write the total file with test_inx in it

In [ ]:
test_inx.to_csv('/Users/AAA218/Desktop/ami_sg/Stamp.csv')

In [4]:
new = pd.read_csv('/Users/AAA218/Desktop/ami_sg/Stamp.csv',header = 0, sep=",",index_col=0,infer_datetime_format=True,low_memory=False)

In [ ]:
# set the building id into index(omitted)
test_inx.set_index(test_inx["bldid"], append = True, inplace = True)

In [5]:
new.head()

,mac,status_info,bldid,xcor,ycor,firstlocatedtime,lastlocatedtime,interval
sgtime,,,,,,,,
2016-04-16 00:00:01,e8:50:8b:94:67:3b,PROBING,SDE1-02,1588.27,382.58,2016-04-15T23:56:54.444+0800,2016-04-15T23:57:14.518+0800,20.0
2016-04-16 02:40:01,94:b1:0a:de:9b:ca,PROBING,SDE1-02,1588.27,382.58,2016-04-16T02:38:26.409+0800,2016-04-16T02:38:26.409+0800,0.0
2016-04-16 02:45:01,94:b1:0a:de:9b:ca,PROBING,SDE1-02,1588.27,382.58,2016-04-16T02:38:26.409+0800,2016-04-16T02:38:26.409+0800,0.0
2016-04-16 07:50:01,84:55:a5:70:d1:eb,PROBING,SDE1-02,1582.69,382.87,2016-04-16T07:48:14.620+0800,2016-04-16T07:48:14.620+0800,0.0
2016-04-16 07:55:01,84:55:a5:70:d1:eb,PROBING,SDE1-02,1582.69,382.87,2016-04-16T07:48:14.620+0800,2016-04-16T07:48:14.620+0800,0.0


#### show the sample sizes every sampling buildings

In [41]:
a = test_inx.groupby("bldid").count().describe()
print a

                 mac    status_info           xcor           ycor  \
count      11.000000      11.000000      11.000000      11.000000   
mean   240959.363636  240959.363636  240959.363636  240959.363636   
std    149381.398918  149381.398918  149381.398918  149381.398918   
min      5684.000000    5684.000000    5684.000000    5684.000000   
25%    155983.000000  155983.000000  155983.000000  155983.000000   
50%    202295.000000  202295.000000  202295.000000  202295.000000   
75%    375301.000000  375301.000000  375301.000000  375301.000000   
max    448547.000000  448547.000000  448547.000000  448547.000000   

       firstlocatedtime  lastlocatedtime       interval  
count         11.000000        11.000000      11.000000  
mean      240959.363636    240959.363636  240959.363636  
std       149381.398918    149381.398918  149381.398918  
min         5684.000000      5684.000000    5684.000000  
25%       155983.000000    155983.000000  155983.000000  
50%       202295.000000    202

#### show the size of samples in each interval

In [42]:
test_inx["xcor"].groupby(level=0).count().describe()

count    5136.000000
mean      516.073403
std       232.968263
min        62.000000
25%       339.000000
50%       556.000000
75%       712.000000
max       940.000000
dtype: float64

### Proportion of different status

In [43]:
prob = test_inx[test_inx["status_info"] == "PROBING"].copy()
associate = test_inx[test_inx['status_info'] == 'ASSOCIATED'].copy()
unknown = test_inx[test_inx['status_info'] == 'UNKNOWN'].copy()
print "probing----",prob.shape[0]
print "associated----",associate.shape[0]
print 'unknown----',unknown.shape[0]

probing---- 1066168
associated---- 1584342
unknown---- 43


#### Generate pie chart for status

In [ ]:
import matplotlib.pyplot as plt
labels = 'Probing', 'Associated', 'Unknown'
sizes = [1066168, 1584342, 43]    # 40.2; 59.8; 0
colors = ['yellowgreen', 'lightskyblue' ,'lightcoral']
explode = (0, 0, 0.1)  # explode 1st slice
 
# Plot
plt.pie(sizes, explode=explode, labels=labels, colors=colors,
        autopct='%1.1f%%', shadow=True, startangle=140)
plt.title('Percentages of different status')
plt.axis('equal')
plt.show()

### Examine out of room samples
* input building sizes

In [1]:
dim = {"SDE1-02":[1227.0858,1730.8369],
"SDE1-03":[275.0,394.0],
"SDE1-04":[1065.6272,1503.0952],
"SDE1-05":[1227.0858,1730.8369],
"SDE2-01":[671.634,948.0622],
"SDE2-02":[736.25146,1038.5022],
"SDE2-03":[673.7935,951.3015],
"SDE3-01" :[1214.772,1734.6945],
"SDE3-02": [2018.2332,2882.037],
"SDE3-03" :[1345.4888,1921.358],
"SDE3-04":[683.8,474.7]} #[length,width]=> [ycor,xcor]

* Function apply to each row, check and generate a series (1 means in)

In [2]:
def out_dim(s):
    bld = str(s["bldid"])
    if(0< s["xcor"] <= dim[bld][1] and 0< s["ycor"] <= dim[bld][0]):
            return pd.Series(dict(out=0))
    else: 
            return pd.Series(dict(out=1))

* test sample output

In [48]:
%%time
a = associate[1:20].apply(out_dim,axis=1)
b = associate.ix[1:20,["xcor","ycor","status_info"]]
print b
b['status_info'] = a
print b

                        xcor    ycor status_info
sgtime                                          
2016-04-16 09:55:01  1562.13  383.85  ASSOCIATED
2016-04-16 10:00:02  1562.13  383.85  ASSOCIATED
2016-04-16 10:30:02  1539.80  384.90  ASSOCIATED
2016-04-16 10:30:02  1536.50  385.03  ASSOCIATED
2016-04-16 10:35:01  1539.80  384.89  ASSOCIATED
2016-04-16 10:40:02  1539.86  384.89  ASSOCIATED
2016-04-16 10:40:02  1550.70  384.36  ASSOCIATED
2016-04-16 10:45:01  1539.73  384.90  ASSOCIATED
2016-04-16 10:50:02  1536.52  385.02  ASSOCIATED
2016-04-16 10:50:02  1547.01  384.53  ASSOCIATED
2016-04-16 17:00:02  1582.69  382.87  ASSOCIATED
2016-04-16 17:45:01  1539.58  387.21  ASSOCIATED
2016-04-16 18:20:02  1574.37  383.28  ASSOCIATED
2016-04-16 21:20:01  1566.70  383.64  ASSOCIATED
2016-04-17 00:45:02  1562.13  383.85  ASSOCIATED
2016-04-17 10:50:01  1579.88  383.01  ASSOCIATED
2016-04-17 10:55:01  1579.88  383.01  ASSOCIATED
2016-04-17 12:05:02  1571.72  383.40  ASSOCIATED
2016-04-17 12:10:01 

In [49]:
%%time
p_out = prob.apply(out_dim, axis=1)
a_out = associate.apply(out_dim, axis=1)
u_out = unknown.apply(out_dim,axis=1) #ran about 20 mins

In [6]:
n_out = new.apply(out_dim,axis = 1)

In [11]:
print new.head(),n_out[0:5]

                                    mac status_info    bldid     xcor    ycor  \
sgtime                                                                         
2016-04-16 00:00:01  e8:50:8b:94:67:3b     PROBING  SDE1-02  1588.27  382.58   
2016-04-16 02:40:01  94:b1:0a:de:9b:ca     PROBING  SDE1-02  1588.27  382.58   
2016-04-16 02:45:01  94:b1:0a:de:9b:ca     PROBING  SDE1-02  1588.27  382.58   
2016-04-16 07:50:01  84:55:a5:70:d1:eb     PROBING  SDE1-02  1582.69  382.87   
2016-04-16 07:55:01  84:55:a5:70:d1:eb     PROBING  SDE1-02  1582.69  382.87   

                                 firstlocatedtime  \
sgtime                                              
2016-04-16 00:00:01  2016-04-15T23:56:54.444+0800   
2016-04-16 02:40:01  2016-04-16T02:38:26.409+0800   
2016-04-16 02:45:01  2016-04-16T02:38:26.409+0800   
2016-04-16 07:50:01  2016-04-16T07:48:14.620+0800   
2016-04-16 07:55:01  2016-04-16T07:48:14.620+0800   

                                  lastlocatedtime  interval  
sgti

### Parse the first/last and add the out mark/hrf/hrl
* function for transfer the datetime and add **hrf/hrl** (need datetime & timedelta)

In [7]:
def stt(df):
    fir = df["firstlocatedtime"].copy()
    las = df["lastlocatedtime"].copy()
    f = lambda x : datetime.strptime(x[0:19],'%Y-%m-%dT%H:%M:%S')
    fir1 = fir.map(f)
    las1 = las.map(f)
    hrf = fir1.map(lambda x: x.hour)
    hrl = las1.map(lambda x: x.hour)
    df.loc[:,"firstlocatedtime"]=fir1
    df.loc[:,"lastlocatedtime"]=las1
    df.loc[:,"hrfir"]=hrf
    df.loc[:,"hrlas"]=hrl
    return df

* apply to the __probing__ and __associated__

In [ ]:
prob1 = stt(prob)
assoc1 = stt(associate)
prob1.to_csv('/Users/AAA218/Desktop/ami_sg/probing.csv')
assoc1.to_csv('/Users/AAA218/Desktop/ami_sg/associate.csv')

* apply to the whole and write

In [12]:
new1 = stt(new)
new1.head()

,mac,status_info,bldid,xcor,ycor,firstlocatedtime,lastlocatedtime,interval,hrfir,hrlas
sgtime,,,,,,,,,,
2016-04-16 00:00:01,e8:50:8b:94:67:3b,PROBING,SDE1-02,1588.27,382.58,2016-04-15 23:56:54,2016-04-15 23:57:14,20.0,23,23
2016-04-16 02:40:01,94:b1:0a:de:9b:ca,PROBING,SDE1-02,1588.27,382.58,2016-04-16 02:38:26,2016-04-16 02:38:26,0.0,2,2
2016-04-16 02:45:01,94:b1:0a:de:9b:ca,PROBING,SDE1-02,1588.27,382.58,2016-04-16 02:38:26,2016-04-16 02:38:26,0.0,2,2
2016-04-16 07:50:01,84:55:a5:70:d1:eb,PROBING,SDE1-02,1582.69,382.87,2016-04-16 07:48:14,2016-04-16 07:48:14,0.0,7,7
2016-04-16 07:55:01,84:55:a5:70:d1:eb,PROBING,SDE1-02,1582.69,382.87,2016-04-16 07:48:14,2016-04-16 07:48:14,0.0,7,7


In [15]:
new.loc[:,'out']=n_out
new.head()

,mac,status_info,bldid,xcor,ycor,firstlocatedtime,lastlocatedtime,interval,hrfir,hrlas,out
sgtime,,,,,,,,,,,
2016-04-16 00:00:01,e8:50:8b:94:67:3b,PROBING,SDE1-02,1588.27,382.58,2016-04-15 23:56:54,2016-04-15 23:57:14,20.0,23,23,1
2016-04-16 02:40:01,94:b1:0a:de:9b:ca,PROBING,SDE1-02,1588.27,382.58,2016-04-16 02:38:26,2016-04-16 02:38:26,0.0,2,2,1
2016-04-16 02:45:01,94:b1:0a:de:9b:ca,PROBING,SDE1-02,1588.27,382.58,2016-04-16 02:38:26,2016-04-16 02:38:26,0.0,2,2,1
2016-04-16 07:50:01,84:55:a5:70:d1:eb,PROBING,SDE1-02,1582.69,382.87,2016-04-16 07:48:14,2016-04-16 07:48:14,0.0,7,7,1
2016-04-16 07:55:01,84:55:a5:70:d1:eb,PROBING,SDE1-02,1582.69,382.87,2016-04-16 07:48:14,2016-04-16 07:48:14,0.0,7,7,1


In [16]:
new.to_csv('/Users/AAA218/Desktop/ami_sg/new_all.csv')

In [50]:
%%time
prob_new = pd.concat([prob,p_out],axis=1)
associate_new = pd.concat([associate,a_out],axis=1)
u_new = pd.concat([unknown,u_out],axis=1)
prob_new.drop('status_info',axis = 1, inplace = True)
associate_new.drop('status_info',axis = 1, inplace = True)
u_new.drop('status_info',axis = 1, inplace = True)

CPU times: user 628 ms, sys: 431 ms, total: 1.06 s
Wall time: 1.11 s


In [51]:
a = prob_new.shape[0]-prob_new["out"].sum()
b = associate_new.shape[0]-associate_new["out"].sum()
c = u_new.shape[0]-u_new["out"].sum()
print "PROBING out of range:",a,"     origin:",prob_new.shape[0]
print "ASSOCIATED out of range:",b,"     origin:",associate_new.shape[0]
print "UNKNOWN out of range:",c,"     origin:",u_new.shape[0]

PROBING out of range: 5072      origin: 1066168
ASSOCIATED out of range: 8047      origin: 1584342
UNKNOWN out of range: 0      origin: 43


In [56]:
%%time
prob_new.to_csv('/Users/AAA218/Desktop/ami_sg/prob_new.csv')

CPU times: user 9.43 s, sys: 299 ms, total: 9.73 s
Wall time: 9.85 s


In [57]:
%%time
associate_new.to_csv('/Users/AAA218/Desktop/ami_sg/assoc_new.csv')

CPU times: user 13.8 s, sys: 372 ms, total: 14.2 s
Wall time: 14.3 s


#### Generating fig of percentage of out-range data

In [ ]:
import matplotlib.pyplot as plt
labels = ['in', 'out']
perc = DataFrame({'Probing':[1066168,5072],'Associated':[1584342,8047]},index=labels)
print perc
percnorm = perc.div(perc.sum(1), axis=0)

# Plot
plt.figure();
percnorm.plot.bar(stacked=True)
plt.title('Percentages of out-range in different status')
plt.show()

### Plotting a few days

In [ ]:
import scipy.ndimage as ndi
import matplotlib.pyplot as plt
#px = test_inx.loc['2016-04-16',["xcor"]]
#py = test_inx.loc['2016-04-16',["ycor"]]
#print px[1:3],py[:-5]

#### print for the sample in 4-16 and SDE1_03

In [ ]:
data = np.random.rand(30000,2)*500       ## create random dataset
print data.shape,"dims---",data.ndim,data[1:5]

In [ ]:
img = np.zeros((500,500))                ## blank image
for i in xrange(data.shape[0]):          ## draw pixels
    img[int(data[i,0]), int(data[i,1])] += 1
img = ndi.gaussian_filter(img, (15,15))

In [ ]:
print img.shape

In [ ]:
i,j = np.where(img == img.max())
print i ,j

fig,ax = plt.subplots()
ax.imshow(img)
ax.scatter(i,j)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
xedges = range(int(SDE1_03['length']))
yedges = range(int(SDE1_03['width']))
H = np.histogram2d(px, py, bins=(xedges,yedges), range=[[0,len(xedges)],[0,len(yedges)]])

In [ ]:
    n = 1000
    # data points range
    data_ymin = -2.
    data_ymax = 2.
    data_xmin = -2.
    data_xmax = 2.
    # view area range
    view_ymin = -.5
    view_ymax = .5
    view_xmin = -.5
    view_xmax = .5
    # generate data
    xl = np.random.uniform(data_xmin, data_xmax, n)    
    yl = np.random.uniform(data_ymin, data_ymax, n)
    zl = np.random.uniform(0, 1, n)

    # get visible data points
    xlvis = []
    ylvis = []
    for i in range(0,len(xl)):
        if view_xmin < xl[i] < view_xmax and view_ymin < yl[i] < view_ymax:
            xlvis.append(xl[i])
            ylvis.append(yl[i])

    fig = plt.figure()
    plt1 = fig.add_subplot(221)
    plt1.set_axis_off()
    zd, xe, ye = np.histogram2d(yl, xl, bins=10, range=[[view_ymin, view_ymax],[view_xmin, view_xmax]], normed=True)
    plt.imshow(zd, origin='lower', extent=[view_xmin, view_xmax, view_ymin, view_ymax])

In [ ]:
 plt.show(fig)

In [ ]:
    # plot histogram
    plt1 = fig.add_subplot(221)
    plt1.set_axis_off()
    zd, xe, ye = np.histogram2d(yl, xl, bins=10, range=[[view_ymin, view_ymax],[view_xmin, view_xmax]], normed=True)
    plt.imshow(zd, origin='lower', extent=[view_xmin, view_xmax, view_ymin, view_ymax])
    plt.scatter(xlvis, ylvis)

    # plot density calculated with kdtree
    plt2 = fig.add_subplot(222)
    plt2.set_axis_off()
    xi = np.linspace(view_xmin, view_xmax, 256)
    yi = np.linspace(view_ymin, view_ymax, 256)
    t0 = time.clock()
    zd = grid_density_kdtree(xl, yl, xi, yi, 70)
    plt.title('function of 5 nearest using kdtree\n'+str(time.clock()-t0)+"sec")
    cmap=cm.jet
    A = (cmap(zd/256.0)*255).astype(np.uint8)
    #A[:,:,3] = zd  
    plt.imshow(A , origin='lower', extent=[view_xmin, view_xmax, view_ymin, view_ymax])
    plt.scatter(xlvis, ylvis)

    # gaussian filter
    plt3 = fig.add_subplot(223)
    plt3.set_axis_off()
    t0 = time.clock()
    zd = grid_density_gaussian_filter(view_xmin, view_ymin, view_xmax, view_ymax, 256, 256, zip(xl, yl))
    plt.title('ndi.gaussian_filter - '+str(time.clock()-t0)+"sec")
    plt.imshow(zd , origin='lower', extent=[view_xmin, view_xmax, view_ymin, view_ymax])
    plt.scatter(xlvis, ylvis)

    # boxsum smoothing
    plt3 = fig.add_subplot(224)
    plt3.set_axis_off()
    t0 = time.clock()
    zd = grid_density_boxsum(view_xmin, view_ymin, view_xmax, view_ymax, 256, 256, zip(xl, yl))
    plt.title('boxsum smoothing - '+str(time.clock()-t0)+"sec")
    plt.imshow(zd, origin='lower', extent=[view_xmin, view_xmax, view_ymin, view_ymax])
    plt.scatter(xlvis, ylvis)

if __name__=='__main__':
    generate_graph()
    plt.show()

### parse the json and create db

In [ ]:
import ijson
from dateutil import parser as dparser
filename = "/Users/AAA218/Desktop/test1.json"

with open(filename, 'r') as f:
    parser = ijson.parse(f)
    print list(parser)
    for prefix,event,value in parser:
        if (prefix, event) == ("item.created_at.$date" , "string"):
            date_temp = dparser.parse(value)
            days = date_temp.date()
            hms = date_temp.timetz()
        elif (prefix, event) == ("item.Locations.entries.item.macAddress" , "string"):
            mac = value
        elif (prefix, event) == ('item.Locations.entries.item.ssId',"string"):
            ssid = value
        elif (prefix, event) == ('item.Locations.entries.item.isGuestUser','boolean'):
                if (value == False): gustid = 0
                    else gustid = 1
        elif (prefix, event) == ('item.Locations.entries.item.dot11Status', 'string'):
            statusid = value
        elif (prefix, event) == ('item.Locations.entries.item.MapInfo.mapHierarchyString', 'string',):
            statusid = value

In [ ]:
mac_add = parsed['Locations']['entries'][0]['macAddress']
first = parsed['Locations']['entries'][0]['Statistics']['firstLocatedTime']
last = parsed['Locations']['entries'][0]['Statistics']['lastLocatedTime']
print first,last

#### Time parse

In [ ]:
from dateutil import parser
first = '2016-04-16T16:00:02.000+0800'
fir_date = parser.parse(first)
print fir_date.date(),fir_date.time(),fir_date.timetz()
#las_date = parser.parse(last)
#datetime.strptime(first,'%y-%m-%dT%H:%M:%S+%fZ')
#connecting = las_date - fir_date
#period = float(connecting.seconds)/60
#print connecting, period